<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [ ]:
from nbdev.showdoc import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import umap
import scvelo as scv
import FRED
if torch.__version__[:4] == '1.14': # If using pytorch with MPS, use Apple silicon GPU acceleration
    device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.has_mps else "cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)
# sns.set_theme()
%load_ext autoreload
%autoreload 2
# set scvelo figures
scv.set_figure_params()

Using device cpu


# 05d01 scVelo on Single Cell Datasets
> Get scVelo's finest embeddings for its hallmark datasets, as baselines

This is a workflow to embed scVelo's datasets using its own state-of-the-art tools -- so that we can compare them to FRED's embeddings.

For each of the datasets, we'll obtain: 
1. scVelo's UMAP visualizations with projected arrows
2. scVelo's latent time computation on the dataset
Both of these will be saved in np txt format for future use.

In [ ]:
notebook = "05d03 scvelo embeddings of sc"
dataset_name = "pancreas"

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [ ]:
# Parameters
notebook = "05d03 scvelo embeddings of sc.ipynb"
dataset_name = "bone marrow"
run num = 6
output_label = "run6_dataset_name-bonemarrow_runnum-6"


SyntaxError: invalid syntax (1506473811.py, line 4)

# Set Up
Adapted from scVelo's tutorials

In [ ]:
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')  # for beautified visualization

In [ ]:
from FRED.datasets import double_helix, directed_swiss_roll_delayed, directed_sinh_branch, rnavelo, rnavelo_pcs
from FRED.data_processing import dataloader_from_ndarray, ManifoldWithVectorField
from torch.utils.data import DataLoader
import scvelo as scv
# choose correct dataset
if dataset_name == "bone marrow":
    adata = scv.datasets.bonemarrow()
elif dataset_name == "dentategyrus":
    adata = scv.datasets.dentategyrus()
elif dataset_name == "pancreas":
    adata = scv.datasets.pancreas()
elif dataset_name == "dentategyrus_lamanno":
    adata = scv.datasets.dentategyrus_lamanno()
else:
    print(f"Dataset {dataset_name} does not exist")
    raise NotImplementedError

In [ ]:
from FRED.datasets import rnavelo_preprocess, rnavelo, rnavelo_pcs, rnavelo_add_labels
# preprocess
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

In [ ]:
# dynamical recovery
scv.tl.recover_dynamics(adata, n_jobs=1)

scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

scv.tl.latent_time(adata)

In [ ]:
# get umap embedding of data
scv.tl.umap(adata)

In [ ]:
scv.pl.velocity_embedding(adata, basis='umap')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap')

In [ ]:
latent_time = adata.obs['latent_time'].to_numpy()

In [ ]:
scv.tl.velocity_embedding(adata, basis='pca', direct_pca_projection=False)

In [ ]:
labels = rnavelo_add_labels(adata)
X = torch.tensor(adata.obsm["X_pca"].copy())
flows = torch.tensor(adata.obsm["velocity_pca"].copy())

# Save embedding for future use

In [ ]:
adata

In [ ]:
embedded_points = adata.obsm['X_umap']

In [ ]:
embedded_velocities = adata.obsm['velocity_umap']

# Metric Evaluation

In [ ]:
# create unique identifier for notebook
import secrets
import string
alphabet = string.ascii_letters + string.digits
unid = ''.join(secrets.choice(alphabet) for i in range(20))  # for a 20-character password
spread_name = notebook + "_dataset_" + dataset_name
unid

In [ ]:
from FRED.metrics import comprehensive_flow_metrics
silhouette_score, silhouete_score_with_flow, knn_score, neighbor_score, monotone_score = comprehensive_flow_metrics(
    X = X,
    flows = flows,
    labels = labels,
    embedded_points = embedded_points,
    embedded_velocities = embedded_velocities,
    time_labels = latent_time,
    spreadsheet_name=spread_name,
    unid = unid,
)

In [ ]:
import csv
with open(f"results/{spread_name}.csv", 'a') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(
            [unid,
            silhouette_score,
            silhouete_score_with_flow,
            knn_score,
            neighbor_score, 
            monotone_score,
            ])